In [9]:
import os
import pandas as pd
import numpy as np
from pandas import DataFrame,Series
import datetime as dt
#from factor_analyzer import FactorAnalyzer
import warnings
warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, learning_curve, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFECV

# Feast Core acts as the central feature registry
FEAST_CORE_URL = os.getenv('FEAST_CORE_URL', 'localhost:6565')

# Feast Online Serving allows for the retrieval of real-time feature data
FEAST_ONLINE_SERVING_URL = os.getenv('FEAST_ONLINE_SERVING_URL', 'localhost:6566')

# Feast Batch Serving allows for the retrieval of historical feature data
FEAST_HISTORICAL_SERVING_URL = os.getenv('FEAST_HISTORICAL_SERVING_URL', 'localhost:6567')


In [10]:
name = pd.DataFrame(pd.read_csv("name.csv"))

In [11]:
name

,ID,Name,First_Name
0,0,Kevin,Wang
1,1,Andy,Anderson
2,2,Chelsey,Taylor


In [12]:
##Add datatime
name['datetime'] = pd.Series([dt.datetime.now()] * len(name))



In [13]:
from feast import Client, FeatureSet, Entity, ValueType, Feature

import pandas as pd
import numpy as np
from pytz import timezone, utc
from feast import Client, FeatureSet, Entity, ValueType
from feast.serving.ServingService_pb2 import GetOnlineFeaturesRequest
from feast.types.Value_pb2 import Value as Value
from google.protobuf.duration_pb2 import Duration
from datetime import datetime, timedelta
from random import randrange
import random

In [14]:
client = Client(core_url=FEAST_CORE_URL, serving_url=FEAST_ONLINE_SERVING_URL)

In [15]:
## Create Feature set 
fs = FeatureSet(
    "overload",
    features = [Feature(name = "Name", dtype=ValueType.STRING,labels=({"name":"English"})),Feature(name = "First_Name", dtype=ValueType.STRING,labels=({"first_name":"English"}))],
    entities = [Entity(name = "ID", dtype = ValueType.INT64)]
)

#driver_fs = FeatureSet("driver_fs", entities=[driver], features=[total_trips_24h])


In [16]:
client.apply(fs)

RpcError: You are attempting to create a feature First_Name that was previously archived. This isn't allowed. Please create a new feature with a different name.

In [17]:
print(fs)

{
  "spec": {
    "name": "overload",
    "entities": [
      {
        "name": "ID",
        "valueType": "INT64"
      }
    ],
    "features": [
      {
        "name": "Name",
        "valueType": "STRING",
        "labels": {
          "name": "English"
        }
      },
      {
        "name": "First_Name",
        "valueType": "STRING",
        "labels": {
          "first_name": "English"
        }
      }
    ]
  },
  "meta": {}
}


In [18]:
print(client.get_feature_set("overload"))

{
  "spec": {
    "name": "overload",
    "entities": [
      {
        "name": "ID",
        "valueType": "INT64"
      }
    ],
    "features": [
      {
        "name": "Name",
        "valueType": "STRING",
        "labels": {
          "name": "English"
        }
      }
    ],
    "source": {
      "type": "KAFKA",
      "kafkaSourceConfig": {
        "bootstrapServers": "kafka:9092,localhost:9094",
        "topic": "feast-features"
      }
    },
    "project": "default"
  },
  "meta": {
    "createdTimestamp": "2020-08-19T03:11:53Z",
    "status": "STATUS_READY"
  }
}


In [19]:
client.ingest("overload", name)

  0%|          | 0/3 [00:00<?, ?rows/s]

Waiting for feature set to be ready for ingestion...


100%|██████████| 3/3 [00:01<00:00,  2.92rows/s]

Ingestion complete!

Ingestion statistics:
Success: 3/3
Removing temporary file(s)...


'492e678a-4238-3772-9d48-199492b2933a'

In [20]:
client.list_feature_sets()

[default/customer_churn,
 default/customer_transactions,
 default/overload,
 default/test,
 default/test1,
 default/titanic,
 default/titanic_demo,
 default/titanic_new]

In [21]:
fs.features[0]

In [22]:
fs.set_label("3","2")

In [ ]:
fs.labels

In [ ]:
fs.entities

In [ ]:
fs.features[1].dtype

In [ ]:
fs.fields

In [ ]:
feat = fs.features

In [ ]:
print(feat[0].labels)

In [23]:
client.get_statistics("overload","1")

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INVALID_ARGUMENT
	details = "Invalid request. Either provide dataset ids to retrieve statistics over, or a start date and end date."
	debug_error_string = "{"created":"@1598241128.687094246","description":"Error received from peer ipv4:172.18.0.2:6565","file":"src/core/lib/surface/call.cc","file_line":1055,"grpc_message":"Invalid request. Either provide dataset ids to retrieve statistics over, or a start date and end date.","grpc_status":3}"
>